In [1]:
#imports
import pandas as pd
import numpy as np
import acquire
import prepare
import model_functions
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
#getting data from SQL
telco = acquire.get_telco_data()
telco.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [3]:
prepped_telco = prepare.prep_telco(telco)

In [4]:
#spliting data
train, validate, test = prepare.split_data(prepped_telco)

In [5]:
customer_id = test.customer_id

In [6]:
customer_id

3160    4531-AUZNK
1577    2296-DKZFP
6823    9692-TUSXH
3615    5129-JLPIS
882     1273-MTETI
           ...    
1097    1597-FZREH
6441    9117-SHLZX
5985    8441-SHIPE
2452    3511-BFTJW
983     1428-GTBJJ
Name: customer_id, Length: 1409, dtype: object

In [7]:
#isolate target variable
def isolate_target():
    df_telco = acquire.get_telco_data()
    df_telco = prepare.prep_telco(df_telco)
    train, validate, test = prepare.split_data(df_telco)

    X_Train = train.drop(columns = ['churn_Yes'])
    y_Train = train.churn_Yes

    X_val = validate.drop(columns = ['churn_Yes'])
    y_val = validate.churn_Yes

    X_test = test.drop(columns = ['churn_Yes'])
    y_test = test.churn_Yes
    return X_Train, y_Train, X_val, y_val, X_test, y_test

#isolate target variable
df_telco = acquire.get_telco_data()
df_telco = prepare.prep_telco(df_telco)
train, validate, test = prepare.split_data(df_telco)

X_Train = train.drop(columns = ['churn_Yes', 'customer_id'])
y_Train = train.churn_Yes

X_val = validate.drop(columns = ['churn_Yes', 'customer_id'])
y_val = validate.churn_Yes

X_test = test.drop(columns = ['churn_Yes', 'customer_id'])
y_test = test.churn_Yes
    

In [8]:
X_Train, y_Train, X_val, y_val, X_test, y_test = model_functions.isolate_target()  

In [9]:
#running best model
rf = RandomForestClassifier(max_depth=9, min_samples_leaf=3, random_state=123)
rf.fit(X_Train, y_Train)
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:,1]

pd.Series(y_pred)
pd.Series(y_prob)

0       0.132971
1       0.029069
2       0.540055
3       0.484610
4       0.742450
          ...   
1404    0.483820
1405    0.575062
1406    0.003597
1407    0.019730
1408    0.577849
Length: 1409, dtype: float64

In [10]:
y_pred = pd.Series(y_pred)
y_prob = pd.Series(y_prob)

In [11]:
y_pred

0       0
1       0
2       1
3       0
4       1
       ..
1404    0
1405    1
1406    0
1407    0
1408    1
Length: 1409, dtype: uint8

In [12]:
X_test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1409 entries, 3160 to 983
Data columns (total 30 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   senior_citizen                         1409 non-null   int64  
 1   tenure                                 1409 non-null   int64  
 2   monthly_charges                        1409 non-null   float64
 3   total_charges                          1409 non-null   float64
 4   gender_Male                            1409 non-null   uint8  
 5   partner_Yes                            1409 non-null   uint8  
 6   dependents_Yes                         1409 non-null   uint8  
 7   phone_service_Yes                      1409 non-null   uint8  
 8   paperless_billing_Yes                  1409 non-null   uint8  
 9   multiple_lines_No phone service        1409 non-null   uint8  
 10  multiple_lines_Yes                     1409 non-null   uint8  
 11  on

In [13]:
#pd.concat(X_test, axis=0,join='left')

In [14]:
df_pred_prob = pd.DataFrame(y_pred,y_prob)



In [15]:
predictions_csv = pd.DataFrame({
                                'customer id': customer_id,
                                'probability of churn': y_prob,
                                'prediction of churn': y_pred,
                                })

In [16]:
predictions_csv.to_csv('predictions.csv')

In [20]:
customer_id.info()

<class 'pandas.core.series.Series'>
Int64Index: 1409 entries, 3160 to 983
Series name: customer_id
Non-Null Count  Dtype 
--------------  ----- 
1409 non-null   object
dtypes: object(1)
memory usage: 54.3+ KB
